In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision import models,datasets
import matplotlib.pyplot as plt
torch.__version__

'1.5.0'

In [2]:
cat_img=Image.open('./1280px-Felis_silvestris_catus_lying_on_rice_straw.jpg')
transform_224= transforms.Compose([
    transforms.Resize(224), 
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])
cat_img_224=transform_224(cat_img)

In [3]:
net = models.vgg16(pretrained=True)
inputs=cat_img_224[np.newaxis]

In [4]:
out = net(inputs)
_, preds = torch.max(out.data, 1)
preds
label=preds.numpy()[0]
label

287

In [5]:
import sys
sys.path.insert(0, './src/')
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
def rescale_grads(map,gradtype="all"):
    if(gradtype=="pos"):    
        map = (np.maximum(0, map) / map.max())
    elif gradtype=="neg":
        map = (np.maximum(0, -map) / -map.min())
    else:
        map = map - map.min()
        map /= map.max()
    return map

In [7]:
inputs.requires_grad=True # 这句话必须要有，否则会报错
from guided_backprop import GuidedBackprop #这里直接引用写好的方法，在src，目录找想对应的文件
GB=GuidedBackprop(net)
gp_grads=GB.generate_gradients(inputs, label)
gp_grads=np.moveaxis(gp_grads,0,-1)
#我们分别计算三类的gp
ag=rescale_grads(gp_grads,gradtype="all")
pg=rescale_grads(gp_grads,gradtype="pos")
ng=rescale_grads(gp_grads,gradtype="neg")

ModuleNotFoundError: No module named 'guided_backprop'

In [ ]:
plt.imshow(cat_img)

In [ ]:
plt.imshow(ag)

In [ ]:
plt.imshow(ng)

In [ ]:
plt.imshow(ag)

In [ ]:
gag=rgb2gray(ag)
plt.imshow(gag)

In [ ]:
gpg=rgb2gray(pg)
plt.imshow(gpg)

In [ ]:
gng=rgb2gray(ng)
plt.imshow(gng)

In [ ]:
import math
from gradcam import GradCam
from guided_gradcam import guided_grad_cam
from guided_backprop import GuidedBackprop
nlayers=len(net.features._modules.items())-1
print(nlayers) # 打印一下一共有多少层
cam_list=[]
#下面我们循环每一层

for layer in range(nlayers):
    #GradCam
    grad_cam = GradCam(net,target_layer=layer)
    cam = grad_cam.generate_cam(inputs, label)
    
    #GuidedBackprop
    GBP = GuidedBackprop(net)
    guided_grads = GBP.generate_gradients(inputs, label)
    
    # Guided Grad cam
    cam_gb = guided_grad_cam(cam, guided_grads)
    cam_list.append(rgb2gray(np.moveaxis(cam_gb,0,-1)))